# Financial Data Crawling
http://erikrood.com/Python_References/dropping_rows_cols_pandas.html

## 1. Price 
### 01. Real Data
20min Delayed Data

    "d1":"Date Last Trade","n":"Name","f6":"Shares","v":"Volume","a2":"Volume Average Daily",
    "l1":"Price(Last Trade)","p":"Price(D-1)","o":"Open","g":"Low","h":"High",
    "c1":"Change","p2":"Change in Percent"

In [1]:
code = '005930.KS'
from Scrpy.krx_scraping import Yahoo_api
Yahoo_api( code ,['o','h','g','l1','p2','v','f6','d1'])

,Open,High,Low,Price(Last Trade),Change in Percent,Volume,Shares,Date Last Trade,Code
0,2521000.0,2544000.0,2518000.0,2536000.0,+0.16%,93139,97668000,7/18/2017,005930.KS


In [2]:
Yahoo_api('053800.KQ',['o','h','g','l1','p2','v','f6','d1']) # 코스닥은 일부 자료가 미출력

,Open,High,Low,Price(Last Trade),Change in Percent,Volume,Shares,Date Last Trade,Code
0,51800.0,51800.0,51300.0,51800.0,+1.17%,46405,NaN,7/18/2017,053800.KQ


### 02. Price History Data
Delayed Data

In [3]:
from pandas_datareader import get_data_yahoo   # Yahoo 가 그나마 빠르게 업데이트 (당일 4시정도)
get_data_yahoo( code ,'2017-7-1','2017-7-15')

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2017-07-03,2375000.0,2389000.0,2356000.0,2361000.0,2361000.0,136111
2017-07-04,2358000.0,2370000.0,2345000.0,2350000.0,2350000.0,159220
2017-07-05,2341000.0,2384000.0,2339000.0,2379000.0,2379000.0,212805
2017-07-06,2400000.0,2405000.0,2386000.0,2403000.0,2403000.0,218461
2017-07-07,2387000.0,2406000.0,2381000.0,2393000.0,2393000.0,162638
2017-07-10,2425000.0,2445000.0,2416000.0,2433000.0,2433000.0,215855
2017-07-11,2432000.0,2450000.0,2414000.0,2450000.0,2450000.0,194356
2017-07-12,2450000.0,2500000.0,2442000.0,2494000.0,2494000.0,189723
2017-07-13,2504000.0,2547000.0,2502000.0,2528000.0,2528000.0,320683


## 2. Trader  Data
### 01. Trader's Valume Data ( Shinhan )
after close market

In [4]:
#df.index = pd.to_datetime(df.index)
from Scrpy.krx_scraping import shinhan_trader
df = shinhan_trader( code ,3)
print(df)

        Code          날짜     개인    외국인계    기관계     증권    투신   은행     종금  \
0  005930.KS  2017/07/18      0       0      0      0     0    0    0.0   
1  005930.KS  2017/07/17  -2054  -13383   -948  13251    63  114 -130.0   
2  005930.KS  2017/07/14   1547  -13429  -4306   9626  -306   18 -165.0   

       보험      기금       기타  
0     0.0     0.0      0.0  
1 -1846.0  4921.0  16206.0  
2 -3421.0  1370.0  16155.0  


### 02. Top Trader Data (daum)
다음사이트 상위거래원 10 거래원 목록 크롤링

In [1]:
# Daum 코스닥 기업목록 불러오기
# Pandas 만 활용할 때 보다 3 ~ 5배 빨라짐 ^^;
import pandas as pd
codes = pd.read_csv('./Scrpy/28W5KQ.csv',sep=',',encoding='cp949')
codes = list(codes.코드)

In [2]:
from Scrpy.krx_scraping import daum_trader
trader = daum_trader(codes[:200])
print(trader.head(5))

  30 /  200  0 min 11 sec, [Estimate  1 min, 14 sec ]
  60 /  200  0 min 20 sec, [Estimate  1 min,  7 sec ]
  90 /  200  0 min 28 sec, [Estimate  1 min,  3 sec ]
 120 /  200  0 min 36 sec, [Estimate  1 min,  0 sec ]
 150 /  200  0 min 45 sec, [Estimate  1 min,  0 sec ]
 180 /  200  0 min 55 sec, [Estimate  1 min,  1 sec ]
        Code     거래원       매도       매수       순매매
0  006570.KQ    현  대       34        0       -34
1  006570.KQ    대  신        1       34       +33
2  006570.KQ    유진증권        0        1        +1
3  015590.KQ  NH투자증권  169,297    9,328  -159,969
4  015590.KQ    키움증권  101,325  103,298    +1,973


In [22]:
import re
samsung = daum_trader('005930.KS')
samsung.순매매 = [int(re.sub(',','',x)) for x in samsung.순매매]
samsung.sort_values('순매매',ascending=False)

,Code,거래원,매도,매수,순매매
1,005930.KS,한국증권,"5,133","11,814",6681
5,005930.KS,제이피모간,0,"5,799",5799
3,005930.KS,삼 성,"5,729","11,272",5543
7,005930.KS,S K,0,"4,247",4247
9,005930.KS,NH투자증권,0,"1,228",1228
4,005930.KS,모건스탠리,"7,324","6,621",-703
11,005930.KS,씨티그룹,763,0,-763
10,005930.KS,메릴린치,"1,431",0,-1431
8,005930.KS,맥쿼리,"2,046",0,-2046
6,005930.KS,대 우,"5,316","1,580",-3736


## 3. Short Trading Data
### 01. Paxnet 공매도 데이터
공매도 자료 (모든 기업이 다 있는건 아니다)

In [8]:
import pandas as pd
short = pd.read_html("http://paxnet.moneta.co.kr/stock/stockIntro/shortSale/"
                     "shortSaleList.jsp?code="+code[:-3]+"&p_curr_page=1")[0]
print(short.head(4))

            0        1       2      3       4           5
0          일자       종가     거래량   매매비중  차입증권매도         NaN
1      공매도 수량   금액(천원)     NaN    NaN     NaN         NaN
2  2017/07/17  2532000  157263  0.37%     584   1485015.0
3  2017/07/14  2524000  159725  4.32%    6897  17473666.0


## 4. LXML 을 활용한 파싱
https://msdn.microsoft.com/ko-kr/kor-kr/enus/library/ms256471 (Xpath 문법해설서)

### 01. Html Table Parsing
by Xpath

In [9]:
from lxml import etree
# lxml 의 편집 가능한 포맷으로 변환
tree = etree.fromstring('<html><head><title>foo</title></head><body>'
      '<div class="name"><p>foo</p></div><div class="name"><ul><li>bar</li></ul></div></body></html>')
for elem in tree.xpath("//div[@class='name']"):
    print(etree.tostring(elem, pretty_print=True))   # HTML 포맷을 이쁘게 출력

b'<div class="name">\n  <p>foo</p>\n</div>\n'
b'<div class="name">\n  <ul>\n    <li>bar</li>\n  </ul>\n</div>\n'


### 02. lxml inner HTML
Extract inner HTML

https://stackoverflow.com/questions/29128478/how-could-i-get-print-all-inner-html-from-node-which-i-select-using-pythons-l

In [10]:
from lxml import etree
from lxml.html import tostring
from io import StringIO
parser = etree.HTMLParser()

Html = '<foo><bar><div>TEST NODE</div></bar></foo>'
html_tree = etree.parse(StringIO(Html), parser)  # Html ==> StingIO 의 묶음 포맷으로 변환
foo_element = html_tree.xpath("//bar")            # Html.xpath() ==> Html의 부분을 잘라낸다
print(len(foo_element), foo_element[0])

1 <Element bar at 0x7fd868e36188>


In [11]:
for i in foo_element[0]:  
    print(tostring(i))

b'<div>TEST NODE</div>'


### 03. Html Table Parsing
by Xpath

In [12]:
doc = """<TABLE>
<TR><TD><P>Name</P></TD> <TD><P>Fees</P></TD> <TD><P>Awards</P></TD> <TD><P>Total</P></TD>
</TR><TR><TD><P>Tony</P></TD> <TD >7,800</TD> <TD >7</TD> <TD>15,400</TD>
</TR><TR><TD><P>Paul</FONT></P></TD> <TD>7,800</TD> <TD>7</TD> <TD>15,400</TD>
</TR><TR><TD><P>Richard</P></TD> <TD>7,800</TD> <TD>7</TD> <TD>15,400</TD>
</TR></TR></TABLE>"""

In [13]:
import lxml.html
root = lxml.html.fromstring(doc)
root.xpath('//tr/td//text()')

['Name',
 'Fees',
 'Awards',
 'Total',
 'Tony',
 '7,800',
 '7',
 '15,400',
 'Paul',
 '7,800',
 '7',
 '15,400',
 'Richard',
 '7,800',
 '7',
 '15,400']

### 04. Html Table Parsing 2
Double Table using by '.'

In [14]:
doc = """<TABLE><TR>
<TD><P>Name</P></TD> <TD><P>Fees</P></TD> <TD><P>Awards</P></TD> <TD><P>Total</P></TD>
</TR><TR><TD><P>Tony</P></TD> <TD >7,800</TD> <TD >7</TD> <TD>15,400</TD>
</TR><TR><TD><P>Paul</FONT></P></TD> <TD >7,800</TD> <TD >7</TD> <TD>15,400</TD>
</TR><TR><TD><P>Richard</P></TD> <TD >7,800</TD> <TD >7</TD> <TD>15,400</TD>
</TR></TR></TABLE>
<TABLE><TR><TD><P>Name</P></TD> <TD><P>Fees</P></TD> <TD><P>Awards</P></TD> <TD><P>Total</P></TD>
</TR><TR><TD><P>Tony</P></TD> <TD >5,800</TD> <TD>7</TD> <TD>45,400</TD>
</TR><TR><TD><P>Park</FONT></P></TD> <TD>3,800</TD> <TD>7</TD> <TD>35,400</TD>
</TR><TR><TD><P>Royal</P></TD> <TD>6,800</TD> <TD>7</TD> <TD>2  5,400</TD>
</TR></TR></TABLE>"""

In [15]:
import lxml.html
root = lxml.html.fromstring(doc); root
# root.xpath('//tr/td//text()')   # / find, // find_all

<Element div at 0x7fd868c0b2c8>

In [16]:
# 2 tables 이 1 문서내 있을 떈 
# 부모 tag 구별 후  
# 자손에는 '.' 를 XPath 문법 맨 앞에 붙이면, 부모 tag를 기준으로 반복실행 한다.

for tbl in root.xpath('//table'):
    elements = tbl.xpath('.//tr/td//text()')
    print (elements)

['Name', 'Fees', 'Awards', 'Total', 'Tony', '7,800', '7', '15,400', 'Paul', '7,800', '7', '15,400', 'Richard', '7,800', '7', '15,400']
['Name', 'Fees', 'Awards', 'Total', 'Tony', '5,800', '7', '45,400', 'Park', '3,800', '7', '35,400', 'Royal', '6,800', '7', '2  5,400']


In [94]:
import pandas as pd

content = '''
<table>
<tbody>
<tr>
<td>265</td>
<td> <a href="/j/jones03.shtml">Jones</a>Blue</td>
<td >29</td>
</tr>
<tr >
<td>266</td>
<td> <a href="/s/smith01.shtml">Smith</a></td>
<td>34</td>
</tr>
</tbody>
</table>'''

table = fromstring(content)
for df in pd.read_html(content):
    df['refname'] = table.xpath('//tr/td/a/@href')
    df['refname'] = df['refname'].str.extract(r'([^./]+)[.]')
    print(df)

     0          1   2  refname
0  265  JonesBlue  29  jones03
1  266      Smith  34  smith01


/home/markbaum/Python/python36/lib/python3.6/site-packages/ipykernel_launcher.py:23: FutureWarning: currently extract(expand=None) means expand=False (return Index/Series/DataFrame) but in a future version of pandas this will be changed to expand=True (return DataFrame)


### 05. Inner HTML Slicing
Daum News

In [17]:
from lxml import etree
from lxml.html import tostring
from io import StringIO
doc = '<foo><bar><div>TEST NODE</div></bar></foo>'
hparser = etree.HTMLParser()
htree = etree.parse(StringIO(doc), hparser)
foo_element = htree.xpath("//div")

In [18]:
len(foo_element)

1

In [19]:
tostring(foo_element[0])

b'<div>TEST NODE</div>'

In [20]:
from lxml import html
import requests, re
url = 'http://media.daum.net/ranking/empathy'
page = requests.get(url)
tree = html.fromstring(page.content)
page.text[:300]

'\n<!doctype html>\n<html lang="ko">\n<head>\n\n\t<meta charset="utf-8">\n<meta http-equiv="X-UA-Compatible" content="IE=Edge">\n\n\n\n<meta property="og:author" content="Daum 뉴스" />\n<meta property="og:site_name" content="Daum 뉴스" />\n<meta property="og:title" content="랭킹"/>\n<meta property="og:image" content="ht'

In [21]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

'청와대는 17일 박근혜 정부의 청와대가 작성한 것으로 보이는 문건 1361건을 추가 발견했다고 밝혔다. 청와대가 지난 14일 민정수석실 캐비닛에서 발견했다며 문건..'

In [22]:
n=1
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/strong/a/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

'靑 "박근혜 정부 문건 1361건 추가 발견..블랙리스트 등 포함"'

In [23]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0]) 
temp = re.sub("  ","",temp); temp

'청와대는 17일 박근혜 정부의 청와대가 작성한 것으로 보이는 문건 1361건을 추가 발견했다고 밝혔다. 청와대가 지난 14일 민정수석실 캐비닛에서 발견했다며 문건..'

In [24]:
# .list_news2 > li:nth-child(1) > div:nth-child(3) > strong:nth-child(1) > a:nth-child(1)
# //*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/strong/a
# //*[@id="mArticle"]/div[2]/ul[2]/li[2]/div[2]/strong/a

In [25]:
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

'청와대는 17일 박근혜 정부의 청와대가 작성한 것으로 보이는 문건 1361건을 추가 발견했다고 밝혔다. 청와대가 지난 14일 민정수석실 캐비닛에서 발견했다며 문건..'

## 5. Stock 38 new list data

###  http://www.38.co.kr/html/fund/?o=nw 의 테이블 불러오기
/html/body/table[3]/tbody/tr/td/table[1]/tbody/tr/td[1]/table[4]/tbody

### 01. lxml.html 을 활용한 크롤링 
inner html 은 부분에 따라 되다 안되다 한다

In [26]:
import requests
url = "http://www.38.co.kr/html/fund/?o=nw"
response = requests.get(url); 
print(response.encoding)
print(response.text[:100])

euc_kr
<!DOCTYPE HTML PUBLIC "-//W3C//DTD HTML 4.01 Transitional//EN">
<html lang="ko">
<head>
<title>공모


#### Xpath Crawling 1
HTML 에서 필요한 table 의 '회사명' 크롤링 하기

In [28]:
from lxml.html import fromstring
html_code = fromstring(response.text)
#text = html_code.xpath('//table[@summary="신규상장종목"]//td//text()') # 가장 가까운 테이블의 속성으로 구분
comp_text = html_code.xpath('//table[@summary="신규상장종목"]//tr/td/a//text()') # 가장 가까운 테이블의 속성으로 구분
#len(comp_text)
comp_text = [txt  for txt in comp_text  if txt not in '(유가)']
print(len(comp_text))
comp_text[:8]

20


['셀트리온헬스케어', '이즈미디어', '한국스팩5호', '힘스', '브이원텍', '아우딘퓨쳐스', '제일홀딩스', '하나금융스팩9호']

#### Xpath Crawling 2
신규회사의 URL 크롤링 하기

In [29]:
from lxml.html import fromstring
html_code = fromstring(response.text)
#text = html_code.xpath('//table[@summary="신규상장종목"]//td//text()') # 가장 가까운 테이블의 속성으로 구분
text = html_code.xpath('//table[@summary="신규상장종목"]//tr/td/a/@href') # 가장 가까운 테이블의 속성으로 구분
url_link = [url[:-6] + txt[1:]  for txt in text  if txt[:8] in './?o=v&n']
print(len(url_link))
url_link[:4]

20


['http://www.38.co.kr/html/fund/?o=v&no=1227&l=',
 'http://www.38.co.kr/html/fund/?o=v&no=1230&l=',
 'http://www.38.co.kr/html/fund/?o=v&no=1231&l=',
 'http://www.38.co.kr/html/fund/?o=v&no=1228&l=']

In [30]:
new_comp = {}
for i in range(len(comp_text)):
    new_comp[comp_text[i]] = url_link[i]
#new_comp

In [31]:
result = []
for i in range(len(comp_text)):
    new = [comp_text[i],url_link[i]]
    result.append(new)
result[:4]

[['셀트리온헬스케어', 'http://www.38.co.kr/html/fund/?o=v&no=1227&l='],
 ['이즈미디어', 'http://www.38.co.kr/html/fund/?o=v&no=1230&l='],
 ['한국스팩5호', 'http://www.38.co.kr/html/fund/?o=v&no=1231&l='],
 ['힘스', 'http://www.38.co.kr/html/fund/?o=v&no=1228&l=']]

#### Xpath Crawling to DataFrame
테이블을 DataFrame으로 출력하기

In [111]:
import re
response = requests.get(result[3][1])
# 기업개요 table 불러오기
text = html_code.xpath('//table[@summary="기업개요"]')
inner_html = tostring(text[0],encoding='unicode')
pd.read_html(inner_html)[0]

,0,1,2,3
0,종목명,힘스,진행상황,공모주
1,시장구분,코스닥,종목코드,238490
2,업종,특수 목적용 기계 제조업,NaN,NaN
3,대표자,김주환,기업구분,중소일반
4,본점소재지,인천광역시 남동구 남동서로 126 (고잔동),NaN,NaN
5,홈페이지,www.hims.co.kr,대표전화,032-821-2511
6,최대주주,-,NaN,NaN
7,매출액,"50,876 (백만원)",법인세비용차감전계속사업이익,"6,115 (백만원)"
8,순이익,"4,951 (백만원)",자본금,"2,072 (백만원)"


In [109]:
# 공모정보 table 불러오기
text = html_code.xpath('//table[@summary="공모정보"]')
inner_html = tostring(text[0],encoding='unicode')
pd.read_html(inner_html)[0]

,0,1,2,3
0,총공모주식수,"800,000 주",액면가,500 원
1,상장공모,"신주모집 : 800,000 주 (100%)",NaN,NaN
2,희망공모가액,"15,500 ~ 18,800 원",청약경쟁률,795.94:1
3,확정공모가,"20,000 원",공모금액,"16,000 (백만원)"
4,주간사,"KB증권,SK증권","주식수: 160,000 주 / 청약한도: - 주",NaN


#### Xpath Crawling to go Detail
개별 회사의 주요내용 크롤링 하기

In [92]:
# 상장코드 번호 찾기
html_code = fromstring(response.text)
text = html_code.xpath('//table[@summary="기업개요"]')
inner_html = tostring(text[0],encoding='unicode')
inner_html = fromstring(inner_html)
text = inner_html.xpath('//tr/td[@bgcolor="#FFFFFF"]/text()')
re.sub('\xa0', '',text[4])

' 238490'

In [93]:
# 주식수 호출하기
text = html_code.xpath('//table[@summary="공모정보"]')
inner_html = tostring(text[0],encoding='unicode')
inner_html = fromstring(inner_html)
text = inner_html.xpath('//tr/td[@bgcolor="#FFFFFF"]/teprint(response.encoding)
print(response.text[:100])xt()')
re.sub('\xa0', '',text[0])

' 800,000 주'

## Daum 상위거래원 크롤링
lxml to List

list to Dataframe

[df1, df2, df3...] concat to DataFrame

In [73]:
code = '008700.KQ'
import requests   # 이게 대부분 포맷을 자동 잡아주나보다 ^^; (euc_kr, utf-8 모두 아래는 동일 코딩적용... )
url = 'http://finance.daum.net/item/trader.daum?code='+code[:-3]+'&nil_profile=stockprice&nil_menu=b015'
response = requests.get(url) 
# print(response.encoding)
# print(response.text[200:500])

In [74]:
from lxml.html import fromstring, tostring
html_code = fromstring(response.text)
texts = html_code.xpath('//table[@id="traderTable"]//tbody/tr/td//text()')

In [75]:
# result = {}
# for i in range(int(len(texts)/4)):
#     result[text[i*4]] = [text[i*4+1],text[i*4+2],text[i*4+3] ]

In [76]:
result = []
for i in range(int(len(texts)/4)):
    result.append([ code, text[i*4], text[i*4+1],text[i*4+2],text[i*4+3] ])

In [44]:
table = html_code.xpath('//table[@id="traderTable"]')
inner_html = tostring(table[0],encoding='unicode')
df = pd.read_html(inner_html)[0]
df = df.iloc[:,:8]
df

,매도상위,매수상위,거래원,매도,매수,순매매,거래원.1,매도.1
0,키움증권,167467,191546.0,"+24,079",키움증권,167467.0,191546.0,"+24,079"
1,대 우,86223,127434.0,"+41,211",대 우,86223.0,127434.0,"+41,211"
2,NH투자증권,46979,40132.0,-6847,현 대,0.0,41754.0,"+41,754"
3,메릴린치,26625,24348.0,-2277,NH투자증권,46979.0,40132.0,-6847
4,하나금융투자,24756,0.0,-24756,삼 성,20015.0,37288.0,"+17,273"
5,이탈 거래원,이탈 거래원,NaN,NaN,NaN,NaN,NaN,NaN
6,한 화,21080,0.0,-21080,메릴린치,26625.0,24348.0,-2277
7,삼 성,20015,37288.0,"+17,273",신한투자,0.0,4555.0,"+4,555"
8,이베스트,16017,0.0,-16017,유안타,1524.0,0.0,-1524
9,IBK투자증권,2600,0.0,-2600,한 화,21080.0,0.0,-21080


## Shinhan trader 자료 크롤링
lxml to List

list to Dataframe

[df1, df2, df3...] concat to DataFrame

In [40]:
code = '008700.KQ'
import requests   # 이게 대부분 포맷을 자동 잡아주나보다 ^^; (euc_kr, utf-8 모두 아래는 동일 코딩적용... )
url = 'http://open.shinhaninvest.com/goodicyber/mk/1206.jsp?code='+code[:-3]
response = requests.get(url) 
# print(response.encoding)
# print(response.text[200:500])

In [42]:
from lxml.html import fromstring, tostring
html_code = fromstring(response.text)
table = html_code.xpath('//table[@class="content1"]')
inner_html = tostring(table[0],encoding='unicode')
df = pd.read_html(inner_html)[0]
df.insert(0,'code',code)
df

,code,0,1,2,3,4,5,6,7,8,9,10
0,008700.KQ,날짜,개인,외국인계,기관,기타,NaN,NaN,NaN,NaN,NaN,NaN
1,008700.KQ,기관계,증권,투신,은행,종금,보험,기금,NaN,NaN,NaN,NaN
2,008700.KQ,2017/07/18,0,0,0,0,0,0,0.0,0.0,0.0,0.0
3,008700.KQ,2017/07/17,79986,-95768,1320,299,0,1021,0.0,0.0,0.0,8640.0
4,008700.KQ,2017/07/14,-299,-3407,1699,1699,0,0,0.0,0.0,0.0,8977.0
5,008700.KQ,2017/07/13,38434,-37518,431,507,0,0,0.0,-76.0,0.0,-27.0
6,008700.KQ,2017/07/12,37207,-39215,209,209,0,0,0.0,0.0,0.0,-160.0
7,008700.KQ,2017/07/11,44060,-42611,-1388,-1388,0,0,0.0,0.0,0.0,-10.0
8,008700.KQ,2017/07/10,102665,-103860,0,0,0,0,0.0,0.0,0.0,32.0
9,008700.KQ,2017/07/07,115132,-116733,-14,-7,-7,0,0.0,0.0,0.0,0.0


In [41]:
row1 = list(df.iloc[0,:]) ; row2 = list(df.iloc[1,:])
row = {0:row1[0], 1:row1[1], 2:row1[2], 3:row2[0], 4:row2[1], 5:row2[2], 
       6:row2[3], 7:row2[4], 8:row2[5], 9:row2[6], 10:row1[4]}
df = df.iloc[2:,:]
df = df.rename(columns=row)
df.reset_index(drop='True', inplace=True); df.head()

,code,008700.KQ,날짜,개인,008700.KQ,기관계,증권,투신,은행,종금,보험,기관
0,008700.KQ,2017/07/18,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,008700.KQ,2017/07/17,79986,-95768,1320,299,0,1021,0.0,0.0,0.0,8640.0
2,008700.KQ,2017/07/14,-299,-3407,1699,1699,0,0,0.0,0.0,0.0,8977.0
3,008700.KQ,2017/07/13,38434,-37518,431,507,0,0,0.0,-76.0,0.0,-27.0
4,008700.KQ,2017/07/12,37207,-39215,209,209,0,0,0.0,0.0,0.0,-160.0


In [39]:
df[df.index<5]

,날짜,개인,외국인계,기관계,증권,투신,은행,종금,보험,기금,기타
0,2017/07/18,0,0,0,0,0,0,0.0,0.0,0.0,0.0
1,2017/07/17,79986,-95768,1320,299,0,1021,0.0,0.0,0.0,8640.0
2,2017/07/14,-299,-3407,1699,1699,0,0,0.0,0.0,0.0,8977.0
3,2017/07/13,38434,-37518,431,507,0,0,0.0,-76.0,0.0,-27.0
4,2017/07/12,37207,-39215,209,209,0,0,0.0,0.0,0.0,-160.0


## PAXNET 공매도

In [14]:
code = '008700.KQ'; no = 1
import requests   # 이게 대부분 포맷을 자동 잡아주나보다 ^^; (euc_kr, utf-8 모두 아래는 동일 코딩적용... )
url = "http://paxnet.moneta.co.kr/stock/stockIntro/shortSale/shortSaleList.jsp?code="+\
       code[:-3]+"&p_curr_page=" + str(no)
response = requests.get(url) 
print(response.encoding)
print(response.text[200:500])

euc-kr
99/xhtml" lang="ko" xml:lang="ko">
<head>
<meta http-equiv="X-UA-Compatible" content="IE=edge">
<meta http-equiv="Content-Type" content="text/html; charset=euc-kr" />
<title>아남전자 공매도 &#45; 팍스넷 No.1 증권포털</title> 
<meta http-equiv="cache-control" content="no-cache" />
<meta http-equiv="pragma" c


In [17]:
from lxml.html import fromstring, tostring
html_code = fromstring(response.text)
table = html_code.xpath('//div[@class="data"]')

In [20]:
inner_html = tostring(table[0],encoding='unicode')
df = pd.read_html(inner_html)[0]; df[df.index<3]

,0,1,2,3,4,5
0,일자,종가,거래량,매매비중,차입증권매도,NaN
1,공매도 수량,금액(천원),NaN,NaN,NaN,NaN
2,2017/07/17,2885,4430491,1.70%,75183,220436.0


In [40]:
row0 = df.iloc[0, :]; result = {}
for i , n in enumerate(row0):
    result[i] = n
result[4] = row1[0]; result[5] = row1[1]
result

{0: '일자', 1: '종가', 2: '거래량', 3: '매매비중', 4: '공매도 수량', 5: '금액(천원)'}

In [43]:
df = df.iloc[2:,:]
df = df.rename(columns=result)
df.head(5)

,일자,종가,거래량,매매비중,공매도 수량,금액(천원)
6,2017/07/11,2910,3259560,0.88%,28566,84126.0
7,2017/07/10,2900,4323386,3.86%,167007,495175.0
8,2017/07/07,3075,4304676,0.54%,23475,73242.0
9,2017/07/06,3150,7216023,0.16%,11616,36718.0
10,2017/07/05,3200,34787799,0.05%,17788,54715.0


In [35]:
row1 = df.iloc[1, :]
row1

0    공매도 수량
1    금액(천원)
2       NaN
3       NaN
4       NaN
5       NaN
Name: 1, dtype: object

In [18]:
df.insert(0,'code',code)
df

,code,0,1,2,3,4,5
0,008700.KQ,일자,종가,거래량,매매비중,차입증권매도,NaN
1,008700.KQ,공매도 수량,금액(천원),NaN,NaN,NaN,NaN
2,008700.KQ,2017/07/17,2885,4430491,1.70%,75183,220436.0
3,008700.KQ,2017/07/14,2830,1451089,0.68%,9902,27923.0
4,008700.KQ,2017/07/13,2840,2114328,0.40%,8524,24284.0
5,008700.KQ,2017/07/12,2830,2397223,2.16%,51820,150744.0
6,008700.KQ,2017/07/11,2910,3259560,0.88%,28566,84126.0
7,008700.KQ,2017/07/10,2900,4323386,3.86%,167007,495175.0
8,008700.KQ,2017/07/07,3075,4304676,0.54%,23475,73242.0
9,008700.KQ,2017/07/06,3150,7216023,0.16%,11616,36718.0


### -------------------------- Under Area is Constructioning....................

### -------------------------- Under Area is Constructioning....................

### -------------------------- Under Area is Constructioning....................

## Selenium

In [ ]:
from selenium import webdriver
driver = webdriver.PhantomJS()

In [ ]:
driver.get('http://example.webscraping.com/search')

In [ ]:
#driver.find_element_by_id('search_term').send_keys('.')

In [ ]:
driver.close()

## Scrapy

In [ ]:
# 스크래피 모듈을 생성
# scrapy startproject example

In [ ]:
# spider 를 작성 
# genspider : (scrapy 내부 수집기 생성)
# country : 스파이더 이름
# example.webscraping.com : 도메인
# --template=crawl : 템플릿

# cd example
# scrapy genspider country example.webscraping.com --template=crawl
# cd example/spider/country.py 에 파이썬 파일이 자동으로 생성

# setting.py 에 봇에 안 걸리도록 설정을 변경 (기본값 8개를 한번에 반복적 수집)
DOWNLOAD_DELAY = 5
CONCURRENT_REQUESTS_PER_DOMAIN = 1

In [ ]:
# scrapy crawl country -s LOG_LEVEL=ERROR  # 스파이더실행 (터미널에서 가능)

In [ ]:
from lxml import html
import requests, re
url = 'http://media.daum.net/ranking/empathy'
page = requests.get(url)
tree = html.fromstring(page.content)
page.text[:300]

In [ ]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

In [ ]:
n=1
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/strong/a/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

In [ ]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0]) 
temp = re.sub("  ","",temp); temp

In [ ]:
# .list_news2 > li:nth-child(1) > div:nth-child(3) > strong:nth-child(1) > a:nth-child(1)
# //*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/strong/a
# //*[@id="mArticle"]/div[2]/ul[2]/li[2]/div[2]/strong/a

In [ ]:
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

## 웹페이지 정보 확인

In [ ]:
import whois
whois.whois('http://used.aladin.co.kr')

## 다운로드 재시도
- 500 오류가 발생시 재시도를 한다

In [ ]:
from urllib.request import urlopen
from urllib.error   import  HTTPError

In [ ]:
def download(url, num_retries = 2):
    print('Downloading: ', url)
    try:
        html = urlopen(url).read().decode('utf-8')
    except HTTPError as e:
        print('Download error: ', e.reason)
        html= None
        if num_retries > 0 :
            if hasattr(e, 'code') and 500 <= e.code <= 600:
                # 5xx HTML 오류시 재시도
                return download(url, num_retries - 1)
    return html

In [ ]:
download('http://httpstat.us/500')

In [ ]:
# 파이선 크롤링 자체를 막는 사이트
download('http://www.samsung.com')

## 웹링크자료 다운로드

In [ ]:
# sitemap_crawler.py
from urllib.request import Request
import re

In [ ]:
def crawl_sitemap(url):
    sitemap = download(url)                          # sitemap 파일 다운로드
    links = re.findall('<loc>(.*?)</loc>', sitemap)  # sitemap 링크 추출 
    for link in links:                               # 추출된 각 링크 다운로드
        html = download(link)                        # 스크랩된 html

In [ ]:
crawl_sitemap('http://example.webscraping.com/sitemap.xml')

## 프록시 지원 접근

In [ ]:
from urllib.request import Request
import urllib.robotparser

In [ ]:
def download(url, user_agent='wswp', proxy=None, num_retries=2):
    print('Downloding:', url)
    headers = {'User-agent': user_agent}
    request = Request(url, headers=headers)
    opener = urllib.request.build_opener()
    if proxy:
        proxy_params = {urllib.request.urlparse(url).scheme: proxy}
        opener.add_handler(urllib.request.ProxyHandler(proxy_params))
    try:
        html = urlopen(request).read().decode('utf-8')
    except HTTPError as e:
        print('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                # 5xx HTTP 오류시 재시도
                return download(url, num_retries - 1)
    return html

In [ ]:
def link_crawler(seed_url, link_regex):
    crawl_queue = [seed_url]      # 이전에 다운로드한 URL 저장
    seen = set(crawl_queue)
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('http://example.webscraping.com/robots.txt')
    rp.read()
    user_agent = 'GoodCrawler'     # 이 아래 user_agent를 번갈아 보면서 실행하면 결과를 알 수 있다
    # user_agent = 'BadCrawler'
    while crawl_queue:
        url = crawl_queue.pop()
        if rp.can_fetch(user_agent, url):
            html = download(url)
            for link in get_links(html):
                if re.match(link_regex, link):   # 링크가 regexd와 일치하는지 확인
                    link = urllib.parse.urljoin(seed_url, link) # 절대 링크 생성
                    if link not in seen:  # 현재 링크가 이전에 다운로드한 링크인지 확인
                        seen.add(link)
                        crawl_queue.append(link)
        else:
            print('Blocked by robots.txt:', seed_url)

In [ ]:
def get_links(html):
    webpage_regex = re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)  # 웹페이지에서 모든 링크를 추출하는 정규식
    return webpage_regex.findall(html)                                       # 웹페이지의 모든 링크 목록

In [ ]:
link_crawler('http://example.webscraping.com', '/(index|view)/')

## 다운로드 조절하기

In [ ]:
# link_crawler_delay.py
import datetime
import time

In [ ]:
class Throttle:
    def __init__(self, delay):
        self.delay = delay # 각 도메인의 다운로드 사이에 지연 시간
        self.domains = {}  # 도메인을 마지막으로 접속한 시간
    def wait(self, url):
        domain = urllib.parse.urlparse(url).netloc
        last_accessed = self.domains.get(domain)
        if self.delay > 0 and last_accessed is not None:
            sleep_secs = self.delay - (datetime.datetime.now() - last_accessed).seconds
            if sleep_secs > 0:          # 도메인을 최근에 접속하였으며, 지연을 할 필요가 있다
                time.sleep(sleep_secs)  # 최근에 접속한 시간 최신화
        self.domains[domain] = datetime.datetime.now()

In [ ]:
def download(url, user_agent='wswp', proxy=None, num_retries=2):
    print('Downloding:', url)
    headers = {'User-agent': user_agent}
    request = Request(url, headers=headers)
    opener = urllib.request.build_opener()
    if proxy:
        proxy_params = {urllib.request.urlparse(url).scheme: proxy}
        opener.add_handler(urllib.request.ProxyHandler(proxy_params))
    try:
        html = urlopen(request).read().decode('utf-8')
    except HTTPError as e:
        print('Download error:', e.reason)
        html = None
        if num_retries > 0:
            if hasattr(e, 'code') and 500 <= e.code < 600:
                return download(url, num_retries - 1)   # 5xx HTTP 오류시 재시도
    return html

In [ ]:
def link_crawler(seed_url, link_regex):
    crawl_queue = [seed_url]
    seen = set(crawl_queue)     # 이전에 다운로드한 URL 저장
    rp = urllib.robotparser.RobotFileParser()
    rp.set_url('http://example.webscraping.com/robots.txt') ; rp.read()
    user_agent = 'GoodCrawler'     # 이 아래 user_agent를 번갈아 보면서 실행하면 결과를 알 수 있다
    # user_agent = 'BadCrawler'
    delay = 2 ; throttle = Throttle(delay)
    while crawl_queue:
        url = crawl_queue.pop()
        if rp.can_fetch(user_agent, url):
            throttle.wait(url) ; html = download(url)
            for link in get_links(html):
                if re.match(link_regex, link):                  # 링크가 regexd와 일치하는지 확인
                    link = urllib.parse.urljoin(seed_url, link) # 절대 링크 생성
                    if link not in seen:                        # 현재 링크가 이전에 다운로드한 링크인지 확인
                        seen.add(link) ; crawl_queue.append(link)
        else:
            print('Blocked by robots.txt:', seed_url)

In [ ]:
def get_links(html):
    webpage_regex = re.compile('<a[^>]+href=["\'](.*?)["\']',re.IGNORECASE)
    return webpage_regex.findall(html)

In [ ]:
link_crawler('http://example.webscraping.com', '/(index|view)/')

# Download

In [ ]:
# hotel_name = tree.xpath(xpath)
# print(len(tree.xpath(xpath)))
# temp = re.sub("\n","",hotel_name[0])
# temp = re.sub("  ","",temp); temp

In [ ]:
from lxml import html
import requests, re
url = 'https://developer.mozilla.org/en-US/docs/Tools/Settings'
page = requests.get(url)
tree = html.fromstring(page.content) ; #page.text[:300]
node = '//*[@id="wikiArticle"]/ul/li[1]/node()'
for txt in tree.xpath(node):
    print('Node :', txt)
text = '//*[@id="wikiArticle"]/ul/li[1]/text()'
print('----------------------------------------------------')
for txt in tree.xpath(text):
    print('Text :', txt)

# MultiThread Download
- https://bitbucket.org/wswp/code/src/9e6b82b47087c2ada0e9fdf4f5e037e151975f0f/chapter04/threaded_test.py?at=default&fileviewer=file-view-default
- https://stackoverflow.com/questions/32285453/why-does-multithreading-do-not-speed-up-parsing-html-with-lxmlm

In [ ]:
from lxml.html import fromstring, HTMLParser
import time
from threading import Thread
from urllib.request import urlopen
def func(number):
    parser = HTMLParser()
    for x in range(number):
        fromstring(DATA, parser=parser)

In [ ]:
DATA = urlopen('http://lxml.de/FAQ.html').read()
print('Testing one thread (100 job per thread)')
start = time.time()
t1 = Thread(target=func, args=[100])
t1.start();    t1.join();    elapsed = time.time() - start
print('Time: %.5f' % elapsed)
print('Testing two threads (50 jobs per thread)')
start = time.time() 
t1 = Thread(target=func, args=[50])
t2 = Thread(target=func, args=[50])
t1.start();   t2.start();   t1.join();  t2.join();  elapsed = time.time() - start
print('Time: %.5f' % elapsed)

## Web Crawling

In [ ]:
# Bs4 01
#url = 'http://example.webscraping.com/view/United Kingdom-239'
#html = download(url)
# soup.find( text = re.compile("sisters") )              # text 내용이 sisters 를 포함시
# soup.find( attrs = {'id' : 'place_area_row'} )
# soup.find( attrs = {'class' : 'w2p_fw'} )

In [ ]:
# Bs4 02 
# soup.find_all( "p")           # <tag>
# soup.find_all( "p", "title" )  # CSS클래스 title 인 <p> tag의 값을 추출
# soup.find_all( id = "link2" )    # id
# soup.find_all( id = True)                              # id 속성을 포함시
# soup.find_all( href = re.compile("elsie") )            # elsie URL 링크 포함시
# soup.find_all( href = re.compile("elsie"), id='link1')

In [ ]:
# Regular Expression 01
# print(re.findall('<td class="w2p_fw">(.*?)</td>', html)[0])
# print(re.findall('<tr id="places_area__row">.*?<td\s*class=["\']w2p_fw["\']>(.*?)</td>', html))
# print(re.findall('<tr id="places_area__row"><td class="w2p_fl"><label for="places_area" id="places_area__label">\
#                   Area: </label></td><td class="w2p_fw">(.*?)</td>', html))

In [ ]:
# Regular Expression 02
# url = 'http://example.webscraping.com/view/United Kingdom-239'
# html = download(url)
# re.findall("<td class='w2p_fw'>(.*?)</td>", html)

In [ ]:
# Regular Expression (제일 빠름)
# results[field] = re.search('<tr id="places_%s__row">.*?<td class="w2p_fw">(.*?)</td>' % field, html).groups()[0]
# lxml  (중간)
# tree = lxml.html.fromstring(html)
# results[field] = tree.cssselect('table > tr#places_%s__row> td.w2p_fw' % field)[0].text_content()
# BS4   (제일 느리다)
# soup = BeautifulSoup(html, 'html.parser')
# results[field] = soup.find('table').find('tr',id='places_%s__row' % field).find('td', class_='w2p_fw').text

## LXML
- C로 만든 XML 분석용 라이브러리
- 설치는 어렵지만 가장 강력한 모듈

In [ ]:
#bs4로 활용찾고, Re로 완성하기
# Regular Expression (용이성은 떨어지지만, 제일 빠르고 파이썬 내부 모듈을 활용) - bs4로 활용찾고, Re로 완성하기
# results[field] = re.search('<tr id="places_%s__row">.*?<td class="w2p_fw">(.*?)</td>' % field, html).groups()[0]

In [ ]:
# find_population_lxml.py
import re
from lxml import html
broken_html = '<ul class=country><li>Area<li>Population</ul>'
tree = html.fromstring(broken_html)  # HTML 분석
fixed_html = html.tostring(tree, pretty_print=True); fixed_html.decode('utf-8')

In [ ]:
from lxml import html
import requests
url = 'http://media.daum.net/ranking/empathy'
page = requests.get(url)
tree = html.fromstring(page.content)
page.text[:300]

In [ ]:
tree.text

In [ ]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

In [ ]:
# .list_news2 > li:nth-child(1) > div:nth-child(3) > strong:nth-child(1) > a:nth-child(1)
# //*[@id="mArticle"]/div[2]/ul[2]/li[1]/div[2]/strong/a
# //*[@id="mArticle"]/div[2]/ul[2]/li[2]/div[2]/strong/a

In [ ]:
n=2

In [ ]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/strong/a/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0])
temp = re.sub("  ","",temp); temp

In [ ]:
xpath = '//*[@id="mArticle"]/div[2]/ul[2]/li['+str(n)+']/div[2]/div/span/text()'
hotel_name = tree.xpath(xpath)
temp = re.sub("\n","",hotel_name[0]) 
temp = re.sub("  ","",temp); temp

## Web Crawling to CSV DB

In [ ]:
from lxml import etree
from io import StringIO
from csv import DictWriter
f= StringIO('''
    <html><body>
    <a class="ui-magnifier-glass" 
       href="here goes the link that i want to extract" data-spm-anchor-id="0.0.0.0" 
       style="width: 258px; height: 258px; position: absolute; left: -1px; top: -1px; display: none;"></a>
    <a href="link to extract" title="title to extract" rel="category tag" data-spm-anchor-id="0.0.0.0">
    or maybe this word instead of title</a>
    </body></html>''')

In [ ]:
doc = etree.parse(f); data=[]
r = doc.xpath('//a[@data-spm-anchor-id="0.0.0.0"]')
for elem in r:
    link=elem.get('href')
    title=elem.get('title')
    text=elem.text
    data.append({'link': link,'title': title,'text': text})

In [ ]:
with open('file.csv', 'w') as csvfile:
    fieldnames=['link', 'title', 'text']
    writer = DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [ ]:
import pandas as pd
pd.read_csv('./file.csv')